### 네이버 뉴스 섹션별 headline 원문 기사 크롤링
- 기사제목, 상세보기, url, 신문사, 작성 날짜 등

In [13]:
import bs4
from bs4 import BeautifulSoup
import warnings
import requests
import pandas as pd
from urllib.request import urlopen
warnings.filterwarnings(action='ignore')
# 중간출력 모두 출력
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [8]:
# 수집할 기사의 url 
head_L_df = pd.read_csv('./crawl_data/naver_news_headline.csv', index_col=0)
head_L_df.tail()

,head_line,news_link,section
47,모나리자 보려면 3만원…한해 900만명 찾는 루브르 입장료 오른다,https://n.news.naver.com/mnews/article/025/000...,세계
48,"WP “이스라엘군이 10월 레바논에서 쓴 백린탄, 美 공급”…美 “확인할 것”",https://n.news.naver.com/mnews/article/081/000...,세계
49,일본 총리 큰일 났네…日국민 10명 중 9명이 내뱉은 충격적인 한마디,https://n.news.naver.com/mnews/article/009/000...,세계
50,"""이스라엘-하마스 인질협상 조만간 재개 가능성""",https://n.news.naver.com/mnews/article/001/001...,세계
51,구글의 올해 검색어 레시피 1위는?...한국의 '비빔밥' [앵커리포트],https://n.news.naver.com/mnews/article/052/000...,세계


### 예시 코드 작성

In [9]:
url = head_L_df['news_link'][50]
url

'https://n.news.naver.com/mnews/article/001/0014386075?sid=104'

In [14]:
# 뉴스에 대한 소스 요청
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'}
res = requests.get(url, headers=headers)
html = res.text
bs_obj = BeautifulSoup(html, 'html.parser')

In [26]:
# 추출하려고 하는 뉴스 기사 영역 한정
content = bs_obj.find("div",{'id':'ct'})
# 기사 작성 신문사
news_com = content.a.img['alt']
# 기사 작성자 및 입력날짜. 
info = content.select('#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans')

In [ ]:
# 복붙해왔을때 가장 끝에있는건 뺴야한다. 여기서는 기사정보 전체를 포함하는것을 info로 일단 불러오고 싶기때문에 그거는 특정 부분의 코드이니까 전체를 포함하고싶기때문
# 뭔말인지 알거임.
#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp

In [ ]:
media_end_head_info_variety_left

In [ ]:
# 기사 입력 및 수정시간/작성기자/좋아요 수/댓글 수
css = '#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp'
#AtributeError: ResultSet object has no attribute 'select' : select() 모듈 사용 불가
#info는 resultset 객체 임(원 소스에서 추출된 data) : select 사용 불가(list 형태)
info1 = info[0].select(css)
info2 = info[0].select('.media_end_head_info_datestamp')
date_all = info2[0].findAll('div',{'class':'media_end_head_info_datestamp_bunch'})
for date in date_all  :
    print(date.text)
ori_date = date_all[0].text.replace('\n','')[2:] 
corr_date = date_all[1].text.replace('\n','')[2:]

In [27]:
# 추천수 복붙해온거는 아래와 같다. 하지만 여기서 마지막 클래스 카운트는 뺴야한다. 이거 가지고 놀아보기.
# 클래스 떼면 무슨일이 있는지? 전과 후 비교 해보기.
#commentFontGroup > div.media_end_head_info_variety_likeit._LIKE_HIDE.as_likeit_improve > div > a > span.u_likeit_text._count.num